In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from src.dataset import SIModel
from src import utils
import seaborn as sns
from scipy.optimize import minimize_scalar, minimize
from scipy.stats import norm, multivariate_normal, probplot


In [ ]:
sns.set_theme()
beta_true = .15
alpha = 0.1
gamma = 0.05
N = 100
T = 52
seed = 29

In [ ]:
F = np.arange(N) % 5
R = np.arange(N) % (N // 2)
fC = utils.contact_matrix(F)
rC = utils.contact_matrix(R)

In [ ]:
X_o = utils.simulator(alpha, beta_true, gamma, N, T, seed)
I_o = X_o.sum(0).astype(int)
plt.plot(I_o / N, label="Infected")
plt.legend(loc="upper right")
plt.xlabel("Time Steps")
plt.ylabel("Proportion")
plt.ylim(0, 1.1)
plt.title("Susceptible-Infected Simulation with Migration")
plt.show()

## Homogeneous Transmission

In [ ]:
def f(beta):
    return - utils.x_loglikelihood(beta, alpha, gamma, N, T, X_o, het=False)

res = minimize_scalar(f, bounds=(0.01,1), method="bounded", options={"disp": True, "maxiter": 18})
res.x

In [ ]:
S = 100
M = - res.fun
sample = np.empty(S)
attempts = 0
np.random.seed(4)
for s in range(S):
    accept = False
    while not accept:
        logbeta = norm(-3, 1).rvs()
        attempts += 1
        u = np.random.uniform(0,1)
        if np.log(u) < utils.x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o) - M:
            accept = True
            sample[s] = logbeta
        if attempts % 1000 == 0:
            print(attempts)
    
sample.mean(), sample.std(), np.exp(sample).mean(), np.exp(sample).std()

In [ ]:
attempts

In [ ]:
# np.random.seed(4)
S = 500
M = - res.fun
sample = np.empty(S)
attempts = 0
np.random.seed(4)
for s in range(S):
    accept = False
    while not accept:
        logbeta = norm(-3, 1).rvs()
        attempts += 1
        u = np.random.uniform(0,1)
        if np.log(u) < utils.x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o) - M:
            accept = True
            sample[s] = logbeta
        if attempts % 1000 == 0:
            print(attempts)
    
sample.mean(), sample.std(), np.exp(sample).mean(), np.exp(sample).std()

In [ ]:
np.exp(sample).std()

In [ ]:
dist = norm(0.137, 0.013)
dist.ppf(.05), dist.ppf(0.95)

In [ ]:
dist = norm(-1.99, 0.0949)
np.exp(dist.ppf(0.05)), np.exp(dist.ppf(0.95))

In [ ]:
plt.hist(np.exp(sample))

In [ ]:
fig, ax = plt.subplots()
probplot(sample, dist="norm", plot=ax)
plt.show()

In [ ]:
# np.random.seed(4)
# S = 100
S = 500
M = - res.fun
sample = np.empty(S)
attempts = 0
np.random.seed(4)
for s in range(S):
    accept = False
    while not accept:
        logbeta = norm(-3, 1).rvs()
        attempts += 1
        u = np.random.uniform(0,1)
        if np.log(u) < x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o) - M:
            accept = True
            sample[s] = logbeta
        if attempts % 1000 == 0:
            print(attempts)
    
sample.mean(), sample.std(), np.exp(sample).mean()

In [ ]:
attempts, np.log10(attempts)

## Identifiability

In [ ]:
def nll(beta, alpha, gamma, N, T, X, het):
    # beta = beta / np.array([1, 300, 300, 300, 300, 300, 300])
    return - utils.x_loglikelihood(beta, alpha, gamma, N, T, X, het)

In [ ]:
res1 = minimize(
    nll, x0 = [0.15, 1e-8, 1e-8, 1e-8, 1e-8, 1e-8, 1e-8], 
    args = (alpha, gamma, N, T, X_o, True),
    bounds = [(0.0, None) for _ in range(7)])
res1.x

In [ ]:
def f2(beta):
    return - x_loglikelihood(beta, alpha, gamma, N, T, X_o, het=True)

res = minimize_scalar(f, bounds=(0.01,1), method="bounded", options={"disp": True, "maxiter": 18})
res.x

## Heterogeneous Transmission

In [ ]:
beta_true = np.array([.05, .02, .04, .06, .08, .1, .05])
alpha = 0.1
gamma = 0.05
heterogeneous = True
N = 300 # consider increasing
T = 52
K = 30
seed = 31

In [ ]:
X_o = utils.simulator(alpha, beta_true, gamma, N, T, seed=31, het=True)
I_o = X_o.sum(0).astype(int)
plt.plot(I_o / N, label="Infected")
plt.legend(loc="upper right")
plt.xlabel("Time Steps")
plt.ylabel("Proportion")
plt.ylim(0, 1.1)
plt.title("Susceptible-Infected Simulation with Migration")
plt.show()

In [ ]:
res2 = minimize(
    utils.nll, x0 = beta_true, args = (alpha, gamma, N, T, X_o, True),
    bounds = [(0.0, None) for _ in range(7)], tol=0.0001
)
res2.x

In [ ]:
np.log(np.array([0.04968622, 0.09937244, 0.31859135, 0.29811733, 0.39748977,
       0.53240004, 5.00131303]))

In [ ]:
res2.fun

In [ ]:
x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o, het=True)

In [ ]:
theta_s = multivariate_normal(prior_mu).rvs()

In [ ]:
prior.logpdf(theta_s)

In [ ]:
def loglike(logbeta):
    return x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o, het=True)

In [ ]:
# metropolis hastings
prior = multivariate_normal(prior_mu)
S = 9000
post_sample = np.empty((S, 7))
theta_s = multivariate_normal(prior_mu).rvs()
sigma = 1
accept = np.empty(S)
post_sample[0] = theta_s
accept[0] = 1
post_cur = prior.logpdf(theta_s) + loglike(theta_s)
u = np.random.uniform(size=S)
np.random.seed(4)
for s in range(1, S):
    theta_t = multivariate_normal(theta_s, sigma).rvs()
    post_prop = prior.logpdf(theta_t) + loglike(theta_t)
    log_A = post_prop - post_cur
    if np.log(u[s]) <= log_A:
        theta_s = theta_t
        post_cur = post_prop
        accept[s] = True
    else:
        accept[s] = False
    post_sample[s] = theta_s
    
    if s % 1000 == 0: print(s)

In [ ]:
# .0572, .0961, .248, .175, .260, .502, 5.21

#### rejection sampling

In [ ]:
prior_mu = np.array([-3, -3, -3, -3, -3, -3, -3])

S = 100
M = - res2.fun
sample = np.empty((S, 7))
attempts = 0
np.random.seed(4)
for s in range(S):
    accept = False
    while not accept:
        logbeta = multivariate_normal(prior_mu).rvs()
        attempts += 1
        u = np.random.uniform(0,1)
        if np.log(u) < utils.x_loglikelihood(np.exp(logbeta), alpha, gamma, N, T, X_o, True) - M:
            accept = True
            sample[s] = logbeta
        if attempts % 500 == 0:
            print(attempts)
    print(s)
    
sample.mean(0), sample.std(0), np.exp(sample).mean(0)

### load posterior sample

In [ ]:
ps = np.load("posterior_sample.npy")

In [ ]:
np.exp(ps).mean(0)

In [ ]:
np.exp(ps).std(0)